In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import geopandas as gpd
from datetime import date

In [ ]:
#Funciones para descargar automáticamente catálogo
today_date = date.today().strftime('%d%m%Y')
download_directory = os.path.join(os.getcwd(), "catalog_" + today_date)  # Guarda en la carpeta "catalog_fechadehoy" en el directorio actual

def wait_for_downloads(download_path, timeout=60):
    """
    Espera a que todos los archivos en el directorio de descarga se completen.
    
    Parámetros:
    - download_path: str, el directorio donde se espera que se descarguen los archivos.
    - timeout: int, el tiempo máximo en segundos para esperar la descarga.
    
    Retorna True si la descarga se completa antes del tiempo de espera, de lo contrario, False.
    """
    seconds = 0
    while seconds < timeout:
        time.sleep(1)
        files = os.listdir(download_path)
        
        # Verifica si hay archivos en el directorio y si alguno tiene la extensión ".crdownload"
        if any(file.endswith(".crdownload") for file in files):
            seconds += 1
        else:
            # Si no hay archivos ".crdownload", se asume que la descarga ha terminado
            return True
    return False

def set_date_with_js(driver, date_picker_id, date_str):
    """Establece la fecha usando JavaScript para los selectores de fecha, con espera hasta que esté disponible."""
    try:
        # Esperar hasta que el selector de fecha esté presente y sea interactivo
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, date_picker_id))
        )

        # Usar JavaScript para establecer directamente el valor de la fecha
        driver.execute_script(f"document.getElementById('{date_picker_id}').value = '{date_str}';")
        
        print(f"Date {date_str} set successfully using JavaScript for date picker with ID '{date_picker_id}'.")

    except Exception as e:
        print(f"Error setting the date via JavaScript for date picker with ID '{date_picker_id}': {e}")
        
        
def automate_seismic_data_submission(initial_date, final_date, 
                             change_longitude=None, 
                             change_latitude=None, 
                             change_magnitude=None, 
                             change_depth=None):
    """
    Automatiza el envío de datos y descarga el archivo Excel.

    Parámetros:
    - initial_date: str, La fecha inicial para establecer en el primer selector de fecha (formato: "DD/MM/YYYY").
    - final_date: str, La fecha final para establecer en el segundo selector de fecha (formato: "DD/MM/YYYY").
    - change_longitude: dict, Opcional; Diccionario con las claves 'longitud_start' y 'longitud_end'.
    - change_latitude: dict, Opcional; Diccionario con las claves 'latitud_start' y 'latitud_end'.
    - change_magnitude: dict, Opcional; Diccionario con las claves 'magnitud_start' y 'magnitud_end'.
    - change_depth: dict, Opcional; Diccionario con las claves 'depth_start' y 'depth_end'.
    """
    # Configuración de Selenium
    
    # Crear el directorio de descarga si no existe
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)
    
    # Configuración de Selenium con opciones de descarga
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory":   download_directory,  # Establece el directorio de descarga
        "download.prompt_for_download": False,             # Desactiva el cuadro de diálogo de descarga
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,                       # Habilita la navegación segura
        "safebrowsing.disable_download_protection": True,   # Desactiva la protección de descargas inseguras
    })

    # Inicializar WebDriver dentro de la función
    driver = webdriver.Chrome(options=chrome_options)

    
    try:
        # Abrir el sitio web
        driver.get("http://bdrsnc.sgc.gov.co/paginas1/catalogo/Consulta_Experta_Seiscomp/consultaexperta.php")

        # Establecer las fechas para los selectores de fecha usando JavaScript
        set_date_with_js(driver,"datepicker", initial_date)  # Establecer la fecha inicial
        set_date_with_js(driver,"datepicker1", final_date)  # Establecer la fecha final

        # Click en el botón de "Parámetros avanzados" para abrir el contenido oculto usando JavaScript
        accordion_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='accordion-titulo' and contains(text(), 'Parámetros avanzados')]"))
        )
        driver.execute_script("arguments[0].click();", accordion_button)

        # Esperar a que el contenido del acordeón esté visible
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "accordion-content"))
        )

        time.sleep(2)

        # Cambiar los valores de longitud si se proporcionan
        if change_longitude:
            longitud_start = driver.find_element(By.ID, "longitudStart")
            longitud_start.clear()
            longitud_start.send_keys(change_longitude.get("longitud_start"))

            longitud_end = driver.find_element(By.ID, "longitudEnd")
            longitud_end.clear()
            longitud_end.send_keys(change_longitude.get("longitud_end"))

        # Cambiar los valores de latitud si se proporcionan
        if change_latitude:
            latitud_start = driver.find_element(By.ID, "latitudStart")
            latitud_start.clear()
            latitud_start.send_keys(change_latitude.get("latitud_start"))

            latitud_end = driver.find_element(By.ID, "latitudEnd")
            latitud_end.clear()
            latitud_end.send_keys(change_latitude.get("latitud_end"))

        # Cambiar los valores de magnitud si se proporcionan
        if change_magnitude:
            magnitud_start = driver.find_element(By.ID, "magnitudStart")
            magnitud_start.clear()
            magnitud_start.send_keys(change_magnitude.get("magnitud_start"))

            magnitud_end = driver.find_element(By.ID, "magnitudEnd")
            magnitud_end.clear()
            magnitud_end.send_keys(change_magnitude.get("magnitud_end"))

        # Cambiar los valores de profundidad si se proporcionan
        if change_depth:
            depth_start = driver.find_element(By.ID, "depthStart")
            depth_start.clear()
            depth_start.send_keys(change_depth.get("depth_start"))

            depth_end = driver.find_element(By.ID, "depthEnd")
            depth_end.clear()
            depth_end.send_keys(change_depth.get("depth_end"))

        # Click en el botón "Consultar" para enviar el formulario usando JavaScript
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@name='Submit' and @value='Consultar']"))
        )
        driver.execute_script("arguments[0].click();", submit_button)

        print("Botón de envío clickeado correctamente.")

        # Esperar a que la nueva página cargue completamente y buscar el enlace "Generar Excel"
        download_link = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'descargar_exel_experta.php') and text()='Generar Excel']"))
        )
        driver.execute_script("arguments[0].click();", download_link)
        #time.sleep(10)
        print("Enlace de descarga de Excel clickeado correctamente. Verifique la carpeta de descargas para el archivo.")
        
        wait_for_downloads(download_directory) # Esperar máximo por 60 segundos la descarga
    except Exception as e:
        print(f"Error durante el proceso de automatización: {e}")

In [ ]:
#importar último catálogo
ultimo_cat = pd.read_excel("D:/cat_03061993_09092024.xlsx", usecols=lambda x: 'Unnamed' not in x)
fecha_min = ultimo_cat.fecha.dt.date.max()
fecha_max = date.today()
fecha_min.strftime('%d/%m/%Y')

In [ ]:
# Ejemplo
automate_seismic_data_submission(
    initial_date=fecha_min.strftime('%d/%m/%Y'),#"01/03/2018",
    final_date=fecha_max.strftime('%d/%m/%Y'),
    change_longitude={"longitud_start": "-77.35", "longitud_end": "-75.80"},
    change_latitude={"latitud_start": "3", "latitud_end": "4.5"},
    change_magnitude={"magnitud_start": "0", "magnitud_end": "8"},
    change_depth={"depth_start": "0", "depth_end": "500"}
)

# Cerrar el navegador
#driver.quit()

In [ ]:
#Antes de correr esta celda, guardar el excel descargado en formato de excel más actualizado
file_name = os.listdir(download_directory)
file_name
new_catalog_2018 = pd.read_excel(download_directory + '\\' + file_name[0], skiprows=13)

In [ ]:
#Renombra columnas respecto a catálogo final
dict_rename_columns_cat_2018 = {'FECHA - HORA UTC': "fecha", 'LATITUD (°)': "lat", 'LONGITUD (°)': "long",
       'PROF. (Km)': "prof", 'MAGNITUD': "mag_u", "TIPO MAGNITUD": "tipo_mag", 'FASES': "fases", 'RMS (Seg)': "rms", 'GAP (°)': "gap",
       'ERROR LATITUD (Km)': "er_lat", 'ERROR LONGITUD (Km)': "er_long", 'ERROR PROFUNDIDAD (Km)':"er_prof",
       'REGION': 'region', 'ESTADO': "estado"}

new_catalog_2018 = new_catalog_2018.rename(columns=dict_rename_columns_cat_2018)

#Definir estructura del catalogo final
out_columns = ["fecha", "dia", "mes", "ano", "lat", "long", "prof", "mag_mi", "mag_mw", "mag_u",
               "tipo_mag", "fases", "rms", "gap", "pais", "dep", "mun", "er_lat", "er_long", "er_prof", "estado", "cat"]

#Crear DataFrame para mejorar estructura
catalog_2018_nst = pd.DataFrame(columns = out_columns)

catalog_2018_nst = catalog_2018_nst.append(new_catalog_2018)

def insertar_diamesano(cat):
    cat.fecha = pd.to_datetime(cat['fecha'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    cat.dia = cat["fecha"].dt.day
    cat.mes = cat["fecha"].dt.month
    cat.ano = cat["fecha"].dt.year
    
#Dividir en columnas municipio, departamento y país en el catálogo 2018
insertar_diamesano(catalog_2018_nst); catalog_2018_nst["cat"] = "2"
catalog_2018_nst[["mun", "pais"]] = catalog_2018_nst["region"].str.split(", ", expand = True)
catalog_2018_nst[["mun", "dep"]] = catalog_2018_nst["mun"].str.split(" - ", expand = True)
catalog_2018_nst.pais = catalog_2018_nst.pais.str.strip(); catalog_2018_nst.mun = catalog_2018_nst.mun.str.strip(); catalog_2018_nst.dep = catalog_2018_nst.dep.str.strip()


In [ ]:
cat_actualizado = pd.concat([catalog_2018_nst, ultimo_cat], axis=0)
#Seleccionar columnas para exportar a shape

In [ ]:
cat_to_shp = cat_actualizado[['fecha', 'lat', 'long', 'prof', 'region', 'mag_u', 'tipo_mag',
        'estado', 'cat']]

geocat = gpd.GeoDataFrame(
    cat_to_shp, geometry=gpd.points_from_xy(cat_to_shp.long, cat_to_shp.lat), crs="EPSG:4326")

#asignar nombre
#Ruta para guardar catálogo unido
ruta = 'D:/'
fecha_min = cat_to_shp['fecha'].dt.date.min().strftime('%d%m%Y')
fecha_max = date.today().strftime('%d%m%Y')
shape_name = "cat_" + fecha_min + "_" + fecha_max 
geocat.to_file(ruta + shape_name, driver='ESRI Shapefile', index = True)
cat_actualizado.to_excel(ruta + shape_name + ".xlsx")